In [1]:
import os
import shutil
import tarfile
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
import pandas as pd
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
import torch
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Intialize the model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)



config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Compile the model with an appropriate optimizer, loss function, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])


In [6]:
model.load_weights(F"/content/gdrive/MyDrive/Colab Notebooks/NLP_ClassProject/3Class/Model_TweetSentimentClassifier_3Class")


In [8]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00


In [9]:
from datasets import load_dataset

dataset = load_dataset("sentiment140")

Generating train split:   0%|          | 0/1600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/498 [00:00<?, ? examples/s]

In [10]:
#Tokenize and encode the data using the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
print(model.summary())

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 109484547 (417.65 MB)
Trainable params: 109484547 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
Review ='''Bahubali is a blockbuster Indian movie that was released in 2015.
It is the first part of a two-part epic saga that tells the story of a legendary hero who fights for his kingdom and his love.
The movie has received rave reviews from critics and audiences alike for its stunning visuals,
spectacular action scenes, and captivating storyline.'''


In [12]:
test = pd.DataFrame.from_dict(dataset['test'])

In [13]:
test = test.drop(columns=['user', 'date', 'query'])

In [30]:
# Function to convert score to sentiment
def to_sentiment(rating):

    rating = int(rating)

    # Convert to class
    if rating < 2:
        return 0
    elif rating == 2:
        return 1
    else:
        return 2


# Apply to the dataset
test['sentiment'] = test.sentiment.apply(to_sentiment)

In [31]:
print(test.head(30))

                                                 text  sentiment  \
0   @stellargirl I loooooooovvvvvveee my Kindle2. ...          2   
1   Reading my kindle2...  Love it... Lee childs i...          2   
2   Ok, first assesment of the #kindle2 ...it fuck...          2   
3   @kenburbary You'll love your Kindle2. I've had...          2   
4   @mikefish  Fair enough. But i have the Kindle2...          2   
5   @richardebaker no. it is too big. I'm quite ha...          2   
6   Fuck this economy. I hate aig and their non lo...          0   
7                       Jquery is my new best friend.          2   
8                                       Loves twitter          2   
9   how can you not love Obama? he makes jokes abo...          2   
10  Check this video out -- President Obama at the...          1   
11  @Karoli I firmly believe that Obama/Pelosi hav...          0   
12  House Correspondents dinner was last night who...          2   
13  Watchin Espn..Jus seen this new Nike Commeri

In [15]:
def text_cleaning(text):
	soup = BeautifulSoup(text, "html.parser")
	text = re.sub(r'\[[^]]*\]', '', soup.get_text())
	pattern = r"[^a-zA-Z0-9\s,']"
	text = re.sub(pattern, '', text)
	return str(text)


In [16]:
test['Cleaned_sentence'] = test['text'].apply(text_cleaning).tolist()

<ipython-input-15-3c233507cf64>:2: MarkupResemblesLocatorWarning:

The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.



In [34]:

# Test data
#test_reviews = "[CLS] " +test_df['Cleaned_sentence'] + "[SEP]"
test_reviews = test['Cleaned_sentence']
test_targets = test['sentiment']


In [38]:
x_val, x_test, y_val, y_test = train_test_split(test_reviews,
													test_targets,
													test_size=0.5,
													stratify = test_targets)

In [19]:
print(test_reviews)

0      stellargirl I loooooooovvvvvveee my Kindle2 No...
1      Reading my kindle2  Love it Lee childs is good...
2      Ok, first assesment of the kindle2 it fucking ...
3      kenburbary You'll love your Kindle2 I've had m...
4      mikefish  Fair enough But i have the Kindle2 a...
                             ...                        
493    Ask Programming LaTeX or InDesign submitted by...
494    On that note, I hate Word I hate Pages I hate ...
495    Ahhh back in a real text editing environment I...
496    Trouble in Iran, I see Hmm Iran Iran so far aw...
497    Reading the tweets coming out of Iran The whol...
Name: Cleaned_sentence, Length: 498, dtype: object


In [39]:
max_len= 128
# Tokenize and encode the sentences

X_test_encoded = tokenizer.batch_encode_plus(x_test.tolist(),
											padding=True,
											truncation=True,
											max_length = max_len,
											return_tensors='tf')

In [40]:
#Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(
	[X_test_encoded['input_ids'], X_test_encoded['token_type_ids'], X_test_encoded['attention_mask']],
	y_test
)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

8/8 [==============================] - 44s 5s/step - loss: 5.1775 - accuracy: 0.6145
Test loss: 5.1775288581848145, Test accuracy: 0.6144578456878662


In [ ]:
input_ids = tf.constant(tokenizer.encode(test_reviews, add_special_tokens=True))[None, :]  # Batch size 1
outputs = model(input_ids)
scores = outputs[0]

In [ ]:
print(input_ids)

tf.Tensor(
[[101 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
  100 100 100 100 100 100 1

In [ ]:
tf_prediction = tf.nn.softmax(scores[0], axis=0).numpy()

In [ ]:
print(tf_prediction)

[0.02445981 0.9755402 ]


In [ ]:
def my_eval(text):
  prediction_vals = []
  for text in text:
    input_ids = tf.constant(tokenizer.encode(text, add_special_tokens=True))[None, :]  # Batch size 1
    outputs = model(input_ids)
    scores = outputs[0]
    tf_prediction = tf.nn.softmax(scores[0], axis=0).numpy()
    prediction_vals.append(tf_prediction)

  return prediction_vals


In [ ]:
def my_vals_redistributed(tf_prediction):
  predictions = []
  for i in range(len(tf_prediction)):
    if abs(tf_prediction[i][0] - tf_prediction[i][1]) < 0.05:
      predictions.append(1)
    elif tf_prediction[i][0] > tf_prediction[i][1]:
      predictions.append(0)
    else:
      predictions.append(2)

  return predictions

In [ ]:
predictions_vals = my_eval(test_reviews)

In [ ]:
print(len(predictions_vals))

498


In [ ]:
predictions = my_vals_redistributed(predictions_vals)


In [ ]:
print(predictions)

[2, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 1, 1, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 1, 0, 0, 1, 1, 0, 2, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, 1, 2, 1, 2, 2, 2, 0, 2, 0, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 1, 2, 2, 0, 2, 0, 2, 2, 2, 2, 1, 0, 1, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 0, 

In [ ]:
# @title Default title text
def pred_counts(predictions):
  counts = [0,0,0]
  for i in range(len(predictions)):
    if predictions[i] == 0:
      counts[0] += 1
    elif predictions[i] == 1:
      counts[1] += 1
    else:
      counts[2] += 1
  return counts

In [ ]:
counts = pred_counts(predictions)

In [ ]:
print(counts)

[184, 52, 262]


In [ ]:
print(predictions)

[2, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 1, 1, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 1, 0, 0, 1, 1, 0, 2, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, 1, 2, 1, 2, 2, 2, 0, 2, 0, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 1, 2, 2, 0, 2, 0, 2, 2, 2, 2, 1, 0, 1, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 0, 

In [ ]:
sentiment_counts = test.value_counts('sentiment')

In [ ]:
y_true = test['sentiment'].to_list()
y_pred = predictions
divisor = 2
y_true = [x//divisor for x in y_true]

In [ ]:
print(y_true)


[2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 0, 2, 2, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 1, 1, 1, 0, 0, 0, 2, 1, 1, 0, 1, 2, 2, 2, 0, 2, 1, 2, 1, 0, 0, 2, 2, 2, 1, 1, 1, 0, 2, 2, 1, 1, 2, 0, 0, 2, 0, 2, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 0, 2, 2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 2, 0, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 2, 2, 2, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 0, 2, 2, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 1, 0, 0, 0, 1, 1, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 2, 0, 2, 2, 1, 1, 0, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 0, 0, 0, 0, 0, 

In [ ]:
print(y_pred)

[2, 2, 2, 2, 2, 1, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 1, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 1, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 1, 2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 2, 0, 

In [ ]:
sum(1 for x,y in zip(y_true,y_pred) if x == y) / len(y_true)

0.6124497991967871